# 4. Salting

Implementing salting in the database for password storage involves generation of a unique random value (the salt) for each user, combining it with their password, and securely hashing the result.

!!!Ensure you have a backup of your data before making changes to the database!!!

**Generate a salt:**
- For each new user account, generate a cryptographically secure random salt. The salt should be long enough to provide sufficient randomness (usually at least 16 bytes).
- A programming language's built-in cryptographic random number generator or a dedicated library can be used to generate a salt.

**Combine salt with password:**
- When a user creates an account or updates their password, the salt has to be combined with their password. This is often done by concatenating the salt and the password.
- The combined value has to be used as input for the password hashing function.

**Hash with salt:**
- Apply a strong, secure, and well-established hashing algorithm (e.g., bcrypt, Argon2) to the combined salt and password. Pass the salt and password combination through the hash function.
- The output of this operation is the hashed password, which includes both the hash and the salt.

**Store hash and salt:**
- In the database, store the hashed password and the salt associated with each user. They should be stored in separate columns in the user's record.

**When a user logs in:**
- Retrieve the salt associated with the user's account from the database.
- Combine the entered password with the retrieved salt.
- Hash the combined salt and password and compare it to the stored hash.

### Add salt to existing passwords

In many cases it can occur, that database already has passwords without applied salt, in this case we have to treat these passwords so they are up to date with the new security requirements.

1. Let`s make a new column in our Customers table where we will save the salt:

```sql
ALTER TABLE customers
ADD COLUMN salting varbinary(16);
```

**Varbinary(16)** is a common practice for security purposes i nthe context of password hashing. As a salt is a random value generated for each customer password entry by using fixed-lenght fields, we ensure that all the salt values are of the same lenght. This consistency in length makes it easy to manage and compare salt values across different records in the database. 

2. Generate a unique salt for each user:

```sql
UPDATE customers SET salting= UNHEX(SUBSTRING(SHA2(UUID(), 256), 1, 32));
```

3. Combine the salt and existing password and hash them:

```sql
UPDATE customers SET password = SHA2(CONCAT(salting, password), 256);
```

4. Commit the changes:


```sql
COMMIT;
```

### Creating a Trigger for new customer`s password

Creating a Trigger can help automate the process and ensure that every new customer's password is properly salted and hashed when they are inserted into the `customers` table. 

1. Create Trigger:
- Define a `BEFORE INSERT` trigger that will be executed before a new customer is inserted into the `customers` table. This trigger will generate a unique salt, combine it with the password, hash the salted password, and then update the `NEW` values accordingly.

```sql
DELIMITER //
CREATE TRIGGER hashedPassword
BEFORE INSERT ON customers
FOR EACH ROW
BEGIN
    SET NEW.salt = UNHEX(SUBSTRING(SHA2(UUID(),256),1,32));
    SET NEW.password = SHA2(CONCAT(NEW.salt, NEW.password),256);
END;
//
DELIMITER ;

The trigger calculates the salt and hashed password and updates the `NEW` values accordingly before insertion.


2. Insert new customer:
- When inserting a new customer into the `customers` table, the trigger will automatically generate the salt and hash the password for that customer.

```sql
INSERT INTO customers (username, password) VALUES ('new_user', 'new_user_password');
```

This trigger will ensure that every new customer's password is securely salted and hashed when they are added to the `customers` table.